In [6]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")
ip.run_line_magic("autoreload", "2")

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import ipyutils as util
np.set_printoptions(suppress=True)
sp.init_printing(use_latex=True)

# Machine numbers

In disem modul wird allgemein in Base 10 mit vier Nachkommastellen gearbeitet.

In [11]:
from machine_accuracy import convert
from machine_accuracy import accuracy

num_float = 58.126654
num_scientific = {
    "mantissa": 0.58126654,
    "exponent": 2,
    "base": 10
}

limits = {
    "mantissa_length": 4,
    "exp_min": 0,
    "exp_max": 3,
    "base": 10
}

x = 5.3451
x_err = 5.3454

float_info = {
    "digits": 8,
    "base": 10
}

def func(x):
    return x ** 2 + 2 * x

def derivative(x):
    return 2 * x + 2

util.dis(f'{num_float} as scientific', convert.to_scientific(num_float))
util.dis(f'{num_scientific} as float', convert.to_float(**num_scientific))
print("")
util.dis(f'{limits} range', accuracy.limits(**limits))
util.dis(f'Absolute error of x', accuracy.absolute_error(approximation=x_err, exact=x))
util.dis(f'Relative error of x', accuracy.relative_error(approximation=x_err, exact=x))
util.dis(f'EPS for number with mantissa length {float_info["digits"]}', accuracy.eps(**float_info))
util.dis(f'Condition of func', accuracy.condition(func=func, derivative=derivative, x=x))
util.dis('Lower condition is better!')
util.dis(expr=accuracy.condition(func=func, derivative=derivative, x=x))

58.126654 as scientific: $\left( 0.58126654, \  10, \  2\right)$

{'mantissa': 0.58126654, 'exponent': 2, 'base': 10} as float: $58.126654$

{'mantissa_length': 4, 'exp_min': 0, 'exp_max': 3, 'base': 10} range: $\left( 0.1, \  999.9\right)$

Absolute error of x: $0.000299999999999301$

Relative error of x: $5.6126171633702 \cdot 10^{-5}$

EPS for number with mantissa length 8: $5.0 \cdot 10^{-8}$

Condition of func: $1.72770962954895$

Lower condition is better!

# Fixed-point iteration

The equation must first be transformed into its fixed-point form:

$$f(x) = 0 \rarr F(x) = x$$